In [1]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [1]:
from reader import *
from string_matching_by_word import *
from string_matching_by_spaCy_NP import *
from word_embeddings_by_spaCy_NP import *
from hobbs import entry
from nltk import pos_tag

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
file_number = 'c4'

### Read Document

In [4]:
input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.input'.format(file_number)
key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.key'.format(file_number)

In [5]:
read = ReadInput(input_path)
list_of_sentences = read.getListOfSentences()
full_text = read.getText()

### Get Sentence Dict and Cluster Head Dict

In [6]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

### Get Coref for exact match by word

In [7]:
coref_dict_match_word = getCorefDict_match_word(sentence_dict, cluster_head_dict,90)
# coref_dict_match_word

### Get Coref for threshold match by NP

In [8]:
cluster_head_dict

{'X0': [0, 'disease'],
 'X1': [0, 'transplant'],
 'X2': [1, 'Trypanosoma cruzi'],
 'X3': [2, 'the United States'],
 'X4': [3, 'report'],
 'X5': [3, 'cases'],
 'X6': [3, 'infection'],
 'X7': [3, 'organs'],
 'X8': [3, 'donor'],
 'X9': [4, 'The Centers for Disease Control and Prevention'],
 'X10': [5, 'recipients'],
 'X11': [9, 'Mod.EP']}

In [9]:
coref_dict_match_NP = getCorefDict_match_NP(sentence_dict, cluster_head_dict,80)
# coref_dict_match_NP

In [10]:
coref_dict_match_NP

{'X3': [[the United States, 3, 'the United States', 100]]}

In [11]:
coref_dict_match_word

{'X0': [['disease', 1, 'disease', 100],
  ['disease', 1, 'disease', 100],
  ['disease', 1, 'disease', 100],
  ['disease', 1, 'disease', 100],
  ['disease', 1, 'disease', 100],
  ['disease', 2, 'disease', 100],
  ['disease', 2, 'disease', 100],
  ['disease', 8, 'disease', 100]],
 'X1': [['transplant', 4, 'transplant', 100],
  ['transplant', 7, 'transplants', 95],
  ['transplant', 8, 'transplants', 95]],
 'X4': [['report', 3, 'Report', 100]],
 'X5': [['cases', 8, 'cases', 100]],
 'X6': [['infection', 4, 'infection', 100],
  ['infection', 5, 'Infection', 100]],
 'X7': [['organs', 5, 'organ', 91],
  ['organs', 5, 'organ', 91],
  ['organs', 8, 'organs', 100],
  ['organs', 8, 'organ', 91],
  ['organs', 8, 'organ', 91]],
 'X8': [['donor', 4, 'donors', 91],
  ['donor', 4, 'donors', 91],
  ['donor', 7, 'donor', 100],
  ['donor', 8, 'donor', 100]],
 'X10': [['recipients', 5, 'recipient', 95]]}

### Get Coref for Word Embedding Similarity by NP

In [14]:
# coref_dict_all_sorted = getCorefDict_all_sorted(sentence_dict, cluster_head_dict,0.5)
# coref_dict_max_sentence = getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict,0.5)

In [12]:
# coref_dict_all_sorted_top3 = get_TopN_Matches(coref_dict_all_sorted,3)

### Merging coref_dict_match_NP with coref_dict_all_sorted_top3

In [13]:
def mergeDicts(dict1, dict2):
    
    d1 = dict1.copy()
    d2 = dict2.copy()
    
    d1_keys = d1.keys()
    d2_keys = d2.keys()
    
    for key in d1_keys:
        d1[key] = list(map(lambda x: x[0:3], d1[key]))
    
    for key in d2_keys:
        d2[key] = list(map(lambda x: x[0:3], d2[key]))
    
    for key in d2_keys:
        if(key not in d1_keys):
            d1[key] = d2[key]
        else:
            for val in d2[key]:
                if(val not in d1[key]):
                    d1[key].append(val)
    
    return dict(sorted(d1.items()))

In [14]:
coref_final = mergeDicts(coref_dict_match_word, coref_dict_match_NP)

In [15]:
coref_final

{'X0': [['disease', 1, 'disease'],
  ['disease', 1, 'disease'],
  ['disease', 1, 'disease'],
  ['disease', 1, 'disease'],
  ['disease', 1, 'disease'],
  ['disease', 2, 'disease'],
  ['disease', 2, 'disease'],
  ['disease', 8, 'disease']],
 'X1': [['transplant', 4, 'transplant'],
  ['transplant', 7, 'transplants'],
  ['transplant', 8, 'transplants']],
 'X10': [['recipients', 5, 'recipient']],
 'X3': [[the United States, 3, 'the United States']],
 'X4': [['report', 3, 'Report']],
 'X5': [['cases', 8, 'cases']],
 'X6': [['infection', 4, 'infection'], ['infection', 5, 'Infection']],
 'X7': [['organs', 5, 'organ'],
  ['organs', 5, 'organ'],
  ['organs', 8, 'organs'],
  ['organs', 8, 'organ'],
  ['organs', 8, 'organ']],
 'X8': [['donor', 4, 'donors'],
  ['donor', 4, 'donors'],
  ['donor', 7, 'donor'],
  ['donor', 8, 'donor']]}

### Passing the reference dict to Hobbs

In [16]:
coref_final_with_pro = entry(list_of_sentences, cluster_head_dict, coref_final)

In [17]:
coref_final_with_pro

{'X0': [['disease', 1, 'disease'],
  ['disease', 1, 'disease'],
  ['disease', 1, 'disease'],
  ['disease', 1, 'disease'],
  ['disease', 1, 'disease'],
  ['disease', 2, 'disease'],
  ['disease', 2, 'disease'],
  ['disease', 8, 'disease']],
 'X1': [['transplant', 4, 'transplant'],
  ['transplant', 7, 'transplants'],
  ['transplant', 8, 'transplants']],
 'X10': [['recipients', 5, 'recipient']],
 'X3': [[the United States, 3, 'the United States']],
 'X4': [['report', 3, 'Report']],
 'X5': [['cases', 8, 'cases']],
 'X6': [['infection', 4, 'infection'], ['infection', 5, 'Infection']],
 'X7': [['organs', 5, 'organ'],
  ['organs', 5, 'organ'],
  ['organs', 8, 'organs'],
  ['organs', 8, 'organ'],
  ['organs', 8, 'organ']],
 'X8': [['donor', 4, 'donors'],
  ['donor', 4, 'donors'],
  ['donor', 7, 'donor'],
  ['donor', 8, 'donor']]}

### Adding those Cluster Heads which were not predicted

In [39]:
def addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro):
    for cluster_id, cluster_val in cluster_head_dict.items():
        if(cluster_id not in coref_final_with_pro.keys()):
            coref_final_with_pro[cluster_id] = [['',int(cluster_val[0]),'', None]]
            
    return coref_final_with_pro

In [40]:
coref_final_with_pro = addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro)

In [41]:
coref_final_with_pro

{'X0': [[Early Aluminum Talks, 5, 'early contract talks'],
  [Early Aluminum Talks, 11, 'the early talks']],
 'X1': [[Steel Union, 5, 'The United Steelworkers union'],
  [Steel Union, 8, "the union's chief negotiator"],
  [Steel Union, 9, 'union members'],
  [Steel Union, 12, 'the Aluminum, Brick and Glass Workers union'],
  [Steel Union, 13, 'the two unions']],
 'X10': [[Pittsburgh-based Alcoa,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X11': [[Reynolds, Richmond, Va.,,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X12': [['union', 12, 'union'], ['union', 13, 'unions']],
 'X3': [[ST. LOUIS, 5, 'The United Steelworkers union'],
  [ST. LOUIS, 9, 'the aluminum industry'],
  [ST. LOUIS, 10, 'normal circumstances'],
  [ST. LOUIS, 12, 'the Steelworkers']],
 'X4': [['1986', 8, '1986']],
 'X6': [[our members, 9, 'union members']],
 'X7': [[sacrifices to help the companies over hard times,
   9,
   'the sacrifices']],
 'X8': [[George Becker, , 9, 'Mr. Becker']],
 '

### Removing the Determiners from the start

In [42]:
def removeDetfromStart(coref_final_with_pro):
    for cluster in coref_final_with_pro.keys():
        for i in range(0, len(coref_final_with_pro[cluster])):
            if(len(coref_final_with_pro[cluster][i][2].split(' '))>1):
                ref_word_list = coref_final_with_pro[cluster][i][2].strip().split(' ')
                ref_word_list = list(filter(bool, ref_word_list) )
                check = pos_tag(ref_word_list)
                if len(check) >1:
                    if 'DT' in check[0][1]:
                        sentence = ' '.join(coref_final_with_pro[cluster][i][2].split(' ')[1:])
                        coref_final_with_pro[cluster][i][2] = ' '.join(coref_final_with_pro[cluster][i][2].split(' ')[1:])
                        
    return coref_final_with_pro

In [43]:
coref_final_with_pro = removeDetfromStart(coref_final_with_pro)

In [44]:
coref_final_with_pro

{'X0': [[Early Aluminum Talks, 5, 'early contract talks'],
  [Early Aluminum Talks, 11, 'early talks']],
 'X1': [[Steel Union, 5, 'United Steelworkers union'],
  [Steel Union, 8, "union's chief negotiator"],
  [Steel Union, 9, 'union members'],
  [Steel Union, 12, 'Aluminum, Brick and Glass Workers union'],
  [Steel Union, 13, 'two unions']],
 'X10': [[Pittsburgh-based Alcoa,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X11': [[Reynolds, Richmond, Va.,,
   13,
   'about 21,000 Alcoa and Reynolds employees']],
 'X12': [['union', 12, 'union'], ['union', 13, 'unions']],
 'X3': [[ST. LOUIS, 5, 'United Steelworkers union'],
  [ST. LOUIS, 9, 'aluminum industry'],
  [ST. LOUIS, 10, 'normal circumstances'],
  [ST. LOUIS, 12, 'Steelworkers']],
 'X4': [['1986', 8, '1986']],
 'X6': [[our members, 9, 'union members']],
 'X7': [[sacrifices to help the companies over hard times, 9, 'sacrifices']],
 'X8': [[George Becker, , 9, 'Mr. Becker']],
 'X2': [['', 1, '', None]],
 'X5': [['', 7, 

### Print Output

In [45]:
def printOP(cluster_head_dict, coref_final_with_pro):
    
    for cluster_id, cluster_head_name in cluster_head_dict.items():

        print('<COREF ID="{}">{}</COREF>'.format(cluster_id, cluster_head_name[1]))

        coreferences = coref_final_with_pro[cluster_id]
        list_of_sent_ids = list(map(lambda x: x[1], coreferences))
        sorted_index_sent_ids = [i[0] for i in sorted(enumerate(list_of_sent_ids), key=lambda x:x[1])]
        coreferences = [coreferences[i] for i in sorted_index_sent_ids]

        for coref in coreferences:
            if(coref[0] == ''):
                continue
            print('{{{0}}}'.format(coref[1]) + ' ' + '{' + coref[2] + '}')
        print('\n', end = '')      

In [46]:
# %%capture cap --no-stderr
printOP(cluster_head_dict, coref_final_with_pro)

<COREF ID="X0">Early Aluminum Talks</COREF>
{5} {early contract talks}
{11} {early talks}

<COREF ID="X1">Steel Union</COREF>
{5} {United Steelworkers union}
{8} {union's chief negotiator}
{9} {union members}
{12} {Aluminum, Brick and Glass Workers union}
{13} {two unions}

<COREF ID="X2">2 Firms</COREF>

<COREF ID="X3">ST. LOUIS</COREF>
{5} {United Steelworkers union}
{9} {aluminum industry}
{10} {normal circumstances}
{12} {Steelworkers}

<COREF ID="X4">1986</COREF>
{8} {1986}

<COREF ID="X5">The current pacts with Aluminum Co. of America and Reynolds Metals Co.</COREF>

<COREF ID="X6">our members</COREF>
{9} {union members}

<COREF ID="X7">sacrifices to help the companies over hard times</COREF>
{9} {sacrifices}

<COREF ID="X8">George Becker, </COREF>
{9} {Mr. Becker}

<COREF ID="X9">a strong recovery</COREF>

<COREF ID="X10">Pittsburgh-based Alcoa</COREF>
{13} {about 21,000 Alcoa and Reynolds employees}

<COREF ID="X11">Reynolds, Richmond, Va.,</COREF>
{13} {about 21,000 Alcoa and 

In [387]:
with open('{}.response'.format(file_number),'w') as f:
    f.write(cap.stdout)

### Answer Key

In [103]:
# reader_o = ReadInput(key_path)
# ans = reader_o.getListOfSentences()
# ans